<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/data-science-11-weeks-progg/blob/main/torch_dive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


**Learn the Basics** ||
[Quickstart](quickstart_tutorial.html) ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
[Build Model](buildmodel_tutorial.html) ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Learn the Basics

Authors:
[Suraj Subramanian](https://github.com/suraj813),
[Seth Juarez](https://github.com/sethjuarez/),
[Cassie Breviu](https://github.com/cassieview/),
[Dmitry Soshnikov](https://soshnikov.com/),
[Ari Bornstein](https://github.com/aribornstein/)

Most machine learning workflows involve working with data, creating models, optimizing model
parameters, and saving the trained models. This tutorial introduces you to a complete ML workflow
implemented in PyTorch, with links to learn more about each of these concepts.

We'll use the FashionMNIST dataset to train a neural network that predicts if an input image belongs
to one of the following classes: T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker,
Bag, or Ankle boot.

`This tutorial assumes a basic familiarity with Python and Deep Learning concepts.`


## Running the Tutorial Code
You can run this tutorial in a couple of ways:

- **In the cloud**: This is the easiest way to get started! Each section has a "Run in Microsoft Learn" and "Run in Google Colab" link at the top, which opens an integrated notebook in Microsoft Learn or Google Colab, respectively, with the code in a fully-hosted environment.
- **Locally**: This option requires you to setup PyTorch and TorchVision first on your local machine ([installation instructions](https://pytorch.org/get-started/locally/)). Download the notebook or copy the code into your favorite IDE.


## How to Use this Guide
If you're familiar with other deep learning frameworks, check out the [0. Quickstart](quickstart_tutorial.html) first
to quickly familiarize yourself with PyTorch's API.

If you're new to deep learning frameworks, head right into the first section of our step-by-step guide: [1. Tensors](tensor_tutorial.html).


.. include:: /beginner_source/basics/qs_toc.txt

.. toctree::
   :hidden:


# Imports

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Create Datasets

In [14]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

In [15]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download= True,
    transform = ToTensor()
)

# Wrap datasets in Dataloader

Wrap datasets with dataloader that acts as an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading.

In [16]:
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size)
test_dataloader = DataLoader(test_data, batch_size)

Check dataloader

In [ ]:
# for X,y in test_dataloader:
#   print(X.shape)
#   print(y.shape, y.dtype)

#torch.Size([64, 1, 28, 28]) 64 pics, with one channel(grascale)(rgb has 3), leagth and width is 28,28
#torch.Size([64]) torch.int64  64 labels

# Setup Device

In [18]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f"Using {device} device")

Using cuda device


# Neural Network Class or Model


To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU or MPS if available.

In [24]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#Loss fn and Optimizer

Make loss function and optimizer


**Optimization** is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

In [39]:
loss_fn =  nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-25)

# Train and Test Fn

In [32]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1)*len(X)
      print(f"loss = {loss:>7f}, current{current:>5d}, size:{size:>5d}")

In [33]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batch = len(dataloader)

  model.eval()
  test_loss, correct = 0,0

  with torch.no_grad():
    for X, y in dataloader:
      X,y = X.to(device), y.to(device)

      pred = model(X)

      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()

  test_loss /= num_batch
  correct /= size
  print(f"Acc={(100*correct):>0.1f}, avg loss={test_loss:>8f}")


#Run

In [40]:
epoch = 5
for t in range(epoch):
  print(t+1)
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader, model, loss_fn)


1
loss = 2.297520, current   64, size:60000
loss = 2.302744, current 6464, size:60000
loss = 2.302143, current12864, size:60000
loss = 2.300788, current19264, size:60000
loss = 2.301084, current25664, size:60000
loss = 2.298843, current32064, size:60000
loss = 2.300575, current38464, size:60000
loss = 2.301897, current44864, size:60000
loss = 2.292238, current51264, size:60000
loss = 2.296605, current57664, size:60000
Acc=9.5, avg loss=2.301013
2
loss = 2.297520, current   64, size:60000
loss = 2.302744, current 6464, size:60000
loss = 2.302143, current12864, size:60000
loss = 2.300788, current19264, size:60000
loss = 2.301084, current25664, size:60000
loss = 2.298843, current32064, size:60000
loss = 2.300575, current38464, size:60000
loss = 2.301897, current44864, size:60000
loss = 2.292238, current51264, size:60000
loss = 2.296605, current57664, size:60000
Acc=9.5, avg loss=2.301013
3
loss = 2.297520, current   64, size:60000
loss = 2.302744, current 6464, size:60000
loss = 2.302143,

#Save

In [41]:
torch.save(model.state_dict(), "model.pth")

# Test saved model

In [42]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [43]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Coat", Actual: "T-shirt/top"
